# Phase 1: Physics Baseline with Monte Carlo

## Objectives
1. Load 1-2 years of historical data (yield, GHI, temperature)
2. Calibrate Monte Carlo to match healthy days
3. Plot actual yield vs MC envelope (P10, P50, P90) over time
4. Sanity checks: Do cloudy days fall in lower tail? Do clear days cluster near P50?

In [ ]:
import sys
sys.path.insert(0, '/workspaces/O-M-Monte-Carlo')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from src.data_loader import SolarDataLoader
from src.phase1_physics.monte_carlo import MonteCarloSimulator

# Set random seed for reproducibility
np.random.seed(42)
sns.set_style('darkgrid')

print("Imports successful!")

## Step 1: Load Historical Data

Expected CSV format: `timestamp, yield, ghi, temp`

In [ ]:
# Initialize data loader
loader = SolarDataLoader(data_dir='data')

# TODO: Replace with your actual data file
# df_raw = loader.load_csv('solar_data.csv')
# df_raw = loader.validate_data(df_raw)

# For now, create synthetic demo data
print("Creating synthetic demo data...")
dates = pd.date_range('2023-01-01', '2024-01-01', freq='1H')
n_periods = len(dates)

# Generate realistic solar data
# GHI follows daily pattern with random clouds
hour_of_day = dates.hour
day_of_year = dates.dayofyear

# Base solar pattern (sine curve for day/night)
ghi_base = 1000 * np.maximum(0, np.sin(np.pi * (hour_of_day - 6) / 12))

# Add seasonal variation
seasonal = 0.5 + 0.5 * np.cos(2 * np.pi * day_of_year / 365)
ghi_base = ghi_base * seasonal

# Add cloud noise (30% of days are cloudy)
cloud_factor = np.ones(n_periods)
for i in range(0, n_periods, 24):
    if np.random.random() < 0.3:  # 30% cloudy days
        cloud_factor[i:i+24] = np.random.uniform(0.3, 0.8)

ghi = ghi_base * cloud_factor + np.random.normal(0, 10, n_periods)
ghi = np.maximum(ghi, 0)

# Yield = efficiency * GHI
efficiency = 0.15  # 15% system efficiency
yield_data = efficiency * ghi + np.random.normal(0, 1, n_periods)
yield_data = np.maximum(yield_data, 0)

# Temperature (simple seasonal + daily pattern)
temp = 15 + 10 * np.cos(2 * np.pi * day_of_year / 365) + 5 * np.sin(np.pi * hour_of_day / 12)

df_raw = pd.DataFrame({
    'yield': yield_data,
    'ghi': ghi,
    'temp': temp
}, index=dates)
df_raw.index.name = 'timestamp'

print(f"Data shape: {df_raw.shape}")
print(f"Date range: {df_raw.index.min()} to {df_raw.index.max()}")
print(f"\nFirst few rows:")
print(df_raw.head())

## Step 2: Aggregate to Daily Data

In [ ]:
# Aggregate to daily summaries
df_daily = loader.get_daily_aggregates(df_raw)

print(f"Daily data shape: {df_daily.shape}")
print(f"\nDaily statistics:")
print(df_daily.describe())

# Visualize raw daily data
fig, axes = plt.subplots(3, 1, figsize=(14, 8))

axes[0].plot(df_daily.index, df_daily['yield'], 'o-', alpha=0.6, markersize=3)
axes[0].set_ylabel('Daily Yield')
axes[0].set_title('Daily Solar Generation')
axes[0].grid(True, alpha=0.3)

axes[1].plot(df_daily.index, df_daily['ghi'], 'o-', alpha=0.6, markersize=3, color='orange')
axes[1].set_ylabel('Daily GHI')
axes[1].set_title('Daily Global Horizontal Irradiance')
axes[1].grid(True, alpha=0.3)

axes[2].plot(df_daily.index, df_daily['temp'], 'o-', alpha=0.6, markersize=3, color='red')
axes[2].set_ylabel('Avg Temperature (°C)')
axes[2].set_xlabel('Date')
axes[2].set_title('Daily Average Temperature')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDaily data loaded and visualized.")

## Step 3: Identify and Calibrate on Healthy Days

Healthy days are those with:
- High efficiency (yield/GHI ratio close to expected)
- Smooth generation profile
- No outlier features

In [ ]:
# Compute efficiency ratio
efficiency_ratio = df_daily['yield'] / (df_daily['ghi'] + 1e-6)

# Define "healthy days" as top quartile of efficiency
# (in practice, you'd validate these manually or use domain knowledge)
efficiency_threshold = efficiency_ratio.quantile(0.75)
healthy_mask = efficiency_ratio > efficiency_threshold

print(f"Healthy days identified: {healthy_mask.sum()} out of {len(df_daily)}")
print(f"Efficiency threshold: {efficiency_threshold:.4f}")
print(f"\nHealthy day efficiency range: {efficiency_ratio[healthy_mask].min():.4f} - {efficiency_ratio[healthy_mask].max():.4f}")

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].scatter(df_daily.index[~healthy_mask], efficiency_ratio[~healthy_mask], 
               alpha=0.5, s=20, label='Non-healthy', color='red')
axes[0].scatter(df_daily.index[healthy_mask], efficiency_ratio[healthy_mask], 
               alpha=0.5, s=20, label='Healthy', color='green')
axes[0].axhline(efficiency_threshold, color='blue', linestyle='--', label='Threshold')
axes[0].set_ylabel('Efficiency (yield/GHI)')
axes[0].set_title('Daily Efficiency Ratio - Healthy Day Identification')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Distribution
axes[1].hist(efficiency_ratio[healthy_mask], bins=20, alpha=0.6, label='Healthy', color='green')
axes[1].hist(efficiency_ratio[~healthy_mask], bins=20, alpha=0.6, label='Non-healthy', color='red')
axes[1].axvline(efficiency_threshold, color='blue', linestyle='--', label='Threshold')
axes[1].set_xlabel('Efficiency Ratio')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Efficiency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 4: Calibrate Monte Carlo Simulator

In [ ]:
# Initialize and calibrate Monte Carlo
mc = MonteCarloSimulator(n_simulations=1000, random_seed=42)
mc.calibrate_from_healthy_days(df_daily, healthy_mask)

print("\nMonte Carlo calibration complete.")
print(f"Mean efficiency: {mc.efficiency_mean:.4f}")
print(f"Efficiency std: {mc.efficiency_std:.4f}")
print(f"GHI noise: {mc.ghi_noise_std:.4f}")

## Step 5: Run MC Simulations for All Days

In [ ]:
# Run simulations for each day
mc_results = []

for date, row in df_daily.iterrows():
    # For daily data, GHI is scalar; create single-timepoint array
    ghi_array = np.array([row['ghi']])
    
    # Run MC
    sims = mc.simulate_day(ghi_array)
    
    # Get statistics
    stats = mc.calculate_day_statistics(np.array([row['yield']]), sims)
    stats['date'] = date
    
    mc_results.append(stats)

df_mc = pd.DataFrame(mc_results).set_index('date')
print(f"MC results shape: {df_mc.shape}")
print(f"\nFirst few MC results:")
print(df_mc.head())

## Step 6: Visualize Actual vs MC Envelope

In [ ]:
# Recompute MC percentiles for visualization
p10_values = []
p50_values = []
p90_values = []

for date, row in df_daily.iterrows():
    ghi_array = np.array([row['ghi']])
    sims = mc.simulate_day(ghi_array)
    daily_sims = sims.sum(axis=1)
    
    p10_values.append(np.percentile(daily_sims, 10))
    p50_values.append(np.percentile(daily_sims, 50))
    p90_values.append(np.percentile(daily_sims, 90))

# Plot time series with MC envelope
fig, ax = plt.subplots(figsize=(16, 6))

ax.fill_between(df_daily.index, p10_values, p90_values, alpha=0.3, color='blue', label='MC P10-P90')
ax.plot(df_daily.index, p50_values, '-', color='blue', linewidth=2, label='MC P50 (expected)')
ax.scatter(df_daily.index[healthy_mask], df_daily['yield'][healthy_mask], 
          alpha=0.6, s=30, color='green', label='Healthy days', zorder=3)
ax.scatter(df_daily.index[~healthy_mask], df_daily['yield'][~healthy_mask], 
          alpha=0.6, s=30, color='red', label='Non-healthy days', zorder=3)

ax.set_xlabel('Date')
ax.set_ylabel('Daily Yield')
ax.set_title('Actual Yield vs Monte Carlo Envelope (P10, P50, P90)')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nEnvelope visualization complete.")

## Step 7: Sanity Checks

### Check 1: Do cloudy/unhealthy days fall in lower tail?

In [ ]:
# Compare percentile ranks
healthy_percentiles = df_mc.loc[healthy_mask, 'percentile_rank']
unhealthy_percentiles = df_mc.loc[~healthy_mask, 'percentile_rank']

print("Sanity Check 1: Percentile Distribution")
print("="*50)
print(f"Healthy days percentile rank:")
print(f"  Mean: {healthy_percentiles.mean():.1f}")
print(f"  Median: {healthy_percentiles.median():.1f}")
print(f"  Range: {healthy_percentiles.min():.1f} - {healthy_percentiles.max():.1f}")

print(f"\nUnhealthy days percentile rank:")
print(f"  Mean: {unhealthy_percentiles.mean():.1f}")
print(f"  Median: {unhealthy_percentiles.median():.1f}")
print(f"  Range: {unhealthy_percentiles.min():.1f} - {unhealthy_percentiles.max():.1f}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(healthy_percentiles, bins=20, alpha=0.6, label='Healthy', color='green')
axes[0].hist(unhealthy_percentiles, bins=20, alpha=0.6, label='Unhealthy', color='red')
axes[0].set_xlabel('MC Percentile Rank')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Days by MC Percentile')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot([healthy_percentiles, unhealthy_percentiles], 
                labels=['Healthy', 'Unhealthy'])
axes[1].set_ylabel('MC Percentile Rank')
axes[1].set_title('Percentile Rank Comparison')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Statistical test
from scipy import stats as sp_stats
t_stat, p_val = sp_stats.ttest_ind(healthy_percentiles, unhealthy_percentiles)
print(f"\nt-test: t={t_stat:.3f}, p-value={p_val:.2e}")
if p_val < 0.05:
    print("✓ PASS: Healthy and unhealthy days have significantly different MC percentiles")
else:
    print("✗ FAIL: No significant difference detected")

### Check 2: Do healthy days cluster near P50?

In [ ]:
print("\nSanity Check 2: Healthy Days Near P50")
print("="*50)

# Healthy days should be clustered around 50th percentile
healthy_in_p40_p60 = ((healthy_percentiles >= 40) & (healthy_percentiles <= 60)).sum()
healthy_pct_near_p50 = 100 * healthy_in_p40_p60 / len(healthy_percentiles)

print(f"Healthy days in P40-P60 range: {healthy_in_p40_p60} / {len(healthy_percentiles)} ({healthy_pct_near_p50:.1f}%)")

if healthy_pct_near_p50 > 50:
    print("✓ PASS: Majority of healthy days cluster near P50")
else:
    print("✗ WARNING: Less than 50% of healthy days near P50")
    print("  Consider: Is your healthy day definition correct? Check calibration.")

# Show distribution of healthy days
fig, ax = plt.subplots(figsize=(12, 5))

ax.scatter(df_daily.index[healthy_mask], df_mc.loc[healthy_mask, 'percentile_rank'], 
          alpha=0.6, s=50, color='green', label='Healthy days')
ax.axhline(50, color='blue', linestyle='--', linewidth=2, label='P50 (expected)')
ax.fill_between(df_daily.index[healthy_mask], 40, 60, alpha=0.1, color='green')

ax.set_xlabel('Date')
ax.set_ylabel('MC Percentile Rank')
ax.set_title('Healthy Days: MC Percentile Over Time')
ax.set_ylim([0, 100])
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

Phase 1 is complete. We have:

1. ✓ Loaded 1 year of historical data (yield, GHI, temp)
2. ✓ Calibrated Monte Carlo to healthy days
3. ✓ Generated MC envelope (P10, P50, P90) for all days
4. ✓ Performed sanity checks on unhealthy vs healthy days

### Next Steps (Phase 2):
- Compute shape-based features (yield/GHI ratio, solar noon ratio, asymmetry)
- Compute MC-derived features (percentile, z-score, uncertainty width)
- Visualize feature separation between known-cloudy and known-clear days

In [ ]:
# Save intermediate results for Phase 2
df_daily.to_csv('data/daily_data.csv')
df_mc.to_csv('data/mc_results.csv')
print("Data saved for Phase 2.")